In [1]:
from transformers import PreTrainedTokenizerFast

In [2]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                   bod_token = '</s>',
                                                   eos_token = '</s>',
                                                   pad_token = '<pad>',
                                                   unk_token = '<unk>',
                                                   mask_token = '<mask>')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [3]:
!pip install fastai

In [4]:
import fastai
from fastai.text.all import *
import torch

In [5]:
class TransformersTokenizer(Transform):
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
    def encodes(self, x):
        toks = self.tokenizer.tokenize(x)
        return tensor(self.tokenizer.convert_tokens_to_ids(toks))
    def decodes(self, x):
        return TitledStr(self.tokenizer.decode(x.cpu().numpy()))

In [6]:
class DropOutput(Callback):
    def after_pred(self):
        self.learn.pred = pred[0]

In [7]:
import pickle
import os

In [8]:
model_dir = os.getenv('HOME') + '/aiffel/aiffel_project/Suggestion_of_alternative/soa_model.pkl'

In [9]:
with open(model_dir, 'rb') as f:
    model = pickle.load(f)

In [10]:
prompt = "저는 장애인과 비장애인이 서로 같은 처지에 있는 사람이라고 생각합니다."
prompt_ids = tokenizer.encode(prompt)
inp = tensor(prompt_ids)[None].cuda()
preds = model.generate(inp,
                            max_length=250,
                            pad_token_id=tokenizer.pad_token_id,
                            eos_token_id=tokenizer.eos_token_id,
                            bos_token_id=tokenizer.bos_token_id,
                            repetition_penalty=2.0,
                            use_cache=True)
src_input = tokenizer.decode(preds[0].cpu().numpy())
print(src_input)

저는 장애인과 비장애인이 서로 같은 처지에 있는 사람이라고 생각합니다. 하지만 그런 상황에서 어떻게 행동해야하는지 고민을 합니다. 왜냐하면 장애인들도 감정이 있고 생각이 잘못된 것이 아닙니다. 그리고 그렇게 생각을 하는 사람들도 많이 있기 때문입니다. 그래서 이러한 인식을 개선하기 위해서는 교육이 필요하다고 생각하고 있습니다. 어렸을 때부터 학교에서 교육을 통해 자라난 청소년부터 제대로 된 성 가치관 등을 교육하고 또 커서도 아직은 부족한 상태라고 할 수 있을 것입니다. 그렇기 때문에 어릴때 부터 자연스럽게 주입하는 인식변화 실태조사를 해야한다고 생각하는 바이고 성인이 되어서는 사회에 대한 정보도 잘 알려져야 한다고 가르쳐주어야하며 어려서부터 이런 형태의 폭력을 받았을 시에 안전하다고 알려주고 올바른 행동을 보여줘서 나중에 사회생활로 복귀할수 있도록 도와 주어야된다고 배워 왔습니다. 요즘 학교폭력이 점점 더 난해지고 있는데이에따라 피해자를 도울 방법은 가해자에게 있다고 말을 하고 싶지 않은 것 같기도 하다고 말하면서 2차 가해를 하지 않도록 예방교육을 시키면 좋을거같다.또한 부모님이나 친구들이 모두 다함께 힘을 합시자.이런 교육은 역할극을 바탕으로 해서 이야기를 와 닿아야지만 모두가 그렇듯 한데 어느 정도 선을 넘고 선뜻 나서주지 않는 한명의 아이들이 피해를 입힐까봐 걱정도 되고싶어하지 않을 거 같다.학교폭력예방교육


In [11]:
!pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
!pip install sacrebleu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
from datasets import load_metric
import pandas as pd
import numpy as np

In [14]:
reference=["""저는 장애인과 비장애인이 서로 같은 처지에 있는 사람이라고 생각합니다. 왜냐하면 장애인들도 감정이 있고 생각만 다를 뿐이지 그런 이유 때문에 차별받는 것은 불합리하다고 생각합니다. 우리도 한 순간에 장애인이 될 수도 있고 그리고 입장을 바꿔서 생각할 때 자신이 장애인이라는 그 이유로 차별을 받아야 하나?라고 생각합니다. 이런 부당한 이유로 사회적으로 차별받고 대우받지 못하는 사회의 인식을 개선해야 할 필요가 있다고 생각합니다. 장애인들도 친구들과 어울려서 이야기도 할 수 있고 게임도 같이 할 수 있는데 장애인이라는 이유만으로 차별하는 건 불합리하다고 생각합니다. 그래서 나는 장애인에 대한 인식을 개선하기 위해서는 일반사람들의 인식을 개선해야 한다고 생각합니다. 우선 자신도 어느 한 순간의 사고로 인해 장애인이 될 수 있기 때문에 장에 대한 생각을 계속 머릿속에 경각심을 심어주어야 한다고 생각합니다. 장애인들도 몸만 불편할 뿐이지 우리와 다르지 않은 사람이라는 생각을 계속 머릿속에 심어주는 것입니다. 계속 반복적으로 머릿속에 이러한 인식들을 심어주면 무의식적인 행동을 하더라도 장애인들을 배려해주는 행동을 하게 된다고 생각합니다. 또 장애인들에게도 교육하는 것이 저는 필요하다고 생각합니다. 장애인들은 자신이 가진 장애 때문에 행동 하나하나가 위축되어서 소심해지는 경향이 있습니다. 저는 장애인들이 위축되거나 소심해지지 않았으면 좋겠습니다. 그래서 장애인들에게 당당해지는 그런 교육을 하는 것이 필요하다고 생각합니다. 그리고 우리 일반인들에게도 교육이 필요하다고 생각한다고 합니다. 학교에서나 회사에서 계속해서 장애인들이 겪는 차별과 부당함에 대해 설명해주고 어떻게 하면 그러한 차별과 부당함을 없앨 수 있을지에 대해 토론을 해본다거나 해결책을 논의 해보는 것도 일종의 방법이라고 생각합니다. 따라서 우리는 누구도 평등하고 인간이 누려야 할 권리는 제한하지말고 누려야 한다고 생각합니다."""]

In [15]:
bleu_metric = load_metric("sacrebleu")

bleu_metric.add(
    prediction=src_input, reference=reference)
results = bleu_metric.compute(smooth_method="floor", smooth_value=0)
results["precisions"] = [np.round(p, 2) for p in results["precisions"]]
pd.DataFrame.from_dict(results, orient="index", columns=["Value"])

,Value
score,7.814685
counts,"[52, 23, 12, 9]"
totals,"[173, 172, 171, 170]"
precisions,"[30.06, 13.37, 7.02, 5.29]"
bp,0.706932
sys_len,173
ref_len,233
